## 11-1 데이터집계

#### groupby 메서드 평균값 구하기

In [1]:
# 데이터로드
import pandas as pd
df = pd.read_csv('data/gapminder.tsv', sep ='\t')
df

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
1,Afghanistan,Asia,1957,30.332,9240934,820.853030
2,Afghanistan,Asia,1962,31.997,10267083,853.100710
3,Afghanistan,Asia,1967,34.020,11537966,836.197138
4,Afghanistan,Asia,1972,36.088,13079460,739.981106
...,...,...,...,...,...,...
1699,Zimbabwe,Africa,1987,62.351,9216418,706.157306
1700,Zimbabwe,Africa,1992,60.377,10704340,693.420786
1701,Zimbabwe,Africa,1997,46.809,11404948,792.449960
1702,Zimbabwe,Africa,2002,39.989,11926563,672.038623


In [2]:
# year열을 기준으로 데이터를 그룹화한 다음lifeExp 열의 평균을 구함
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
avg_life_exp_by_year

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

#### 분할-반영-결합 과정 살펴보기

In [3]:
years = df.year.unique()
years

array([1952, 1957, 1962, 1967, 1972, 1977, 1982, 1987, 1992, 1997, 2002,
       2007], dtype=int64)

In [5]:
# 연도별 데이터 추출
y1952 = df.loc[df.year == 1952, :]
y1952.head()

,country,continent,year,lifeExp,pop,gdpPercap
0,Afghanistan,Asia,1952,28.801,8425333,779.445314
12,Albania,Europe,1952,55.230,1282697,1601.056136
24,Algeria,Africa,1952,43.077,9279525,2449.008185
36,Angola,Africa,1952,30.015,4232095,3520.610273
48,Argentina,Americas,1952,62.485,17876956,5911.315053


In [6]:
# lifeExp의 평균
y1952_mean = y1952.lifeExp.mean()
y1952_mean

49.05761971830987

In [10]:
# 위의 2개 과정을 반복하여 연도별 평균값 가져옴
y1957 = df.loc[df.year == 1957, :]
y1957_mean = y1957.lifeExp.mean()
print(y1957_mean)

y1962 = df.loc[df.year == 1962, :]
y1962_mean = y1962.lifeExp.mean()
print(y1962_mean)

y2007 = df.loc[df.year == 2007, :]
y2007_mean = y2007.lifeExp.mean()
print(y2007_mean)

51.50740112676054
53.60924901408449
67.00742253521126


In [11]:
df2 = pd.DataFrame({'year': [1952, 1957, 1962, 2007],
                    "": [y1952_mean, y1957_mean, y1962_mean, y2007_mean]})
df2

,year,
0,1952,49.057620
1,1957,51.507401
2,1962,53.609249
3,2007,67.007423


#### 평균값을 구하는 사용자 함수와 groupby 메서드

In [13]:
# 열의 평균값을 구하는 함수
def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    return sum / n

In [14]:
agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
agg_my_mean

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64

#### 2개의 인잣값을 받아 처리하는 사용자 함수

In [15]:
def my_mean_diff(values, diff_value):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value

In [16]:
global_mean = df.lifeExp.mean()
global_mean

59.47443936619713

In [17]:
agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value = global_mean)
agg_mean_diff

year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64

#### 집계 메서드를 리스트, 딕셔너리에 담아 전달하기

In [18]:
# 연도별로 그룹화한 lifeExp열의 0이 아닌 값의 개수, 평균, 표준편차를 한번에 계산하여 출력
import numpy as np

gdf = df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
gdf

,count_nonzero,mean,std
year,,,
1952,142.0,49.057620,12.225956
1957,142.0,51.507401,12.231286
1962,142.0,53.609249,12.097245
1967,142.0,55.678290,11.718858
1972,142.0,57.647386,11.381953
1977,142.0,59.570157,11.227229
1982,142.0,61.533197,10.770618
1987,142.0,63.212613,10.556285
1992,142.0,64.160338,11.227380


In [20]:
# 집계 메서드를 딕셔너리에 담아 agg 메서드에 전달
gdf_dict = df.groupby('year').agg({'lifeExp' : 'mean', 'pop' : 'median', 'gdpPercap':'median'})
gdf_dict

,lifeExp,pop,gdpPercap
year,,,
1952,49.057620,3943953.0,1968.528344
1957,51.507401,4282942.0,2173.220291
1962,53.609249,4686039.5,2335.439533
1967,55.678290,5170175.5,2678.334741
1972,57.647386,5877996.5,3339.129407
1977,59.570157,6404036.5,3798.609244
1982,61.533197,7007320.0,4216.228428
1987,63.212613,7774861.5,4280.300366
1992,64.160338,8688686.5,4386.085502


## 11-2 데이터 변환

#### 표준점수 계산하기

In [21]:
# 표준점수를 계산하는 함수
def my_zscore(x):
    return (x - x.mean()) / x.std()

In [22]:
# 연도별 lifeExp열의 표준점수를 계산
transform_z = df.groupby('year').lifeExp.transform(my_zscore)
transform_z

0      -1.656854
1      -1.731249
2      -1.786543
3      -1.848157
4      -1.894173
          ...   
1699   -0.081621
1700   -0.336974
1701   -1.574962
1702   -2.093346
1703   -1.948180
Name: lifeExp, Length: 1704, dtype: float64

In [23]:
# my_zscore함수는 데이터를 표준화할뿐, 집계는 하지 않음
print(df.shape)
print(transform_z.shape)

(1704, 6)
(1704,)


#### 누락값을 평균값으로 처리하기

In [26]:
# total_bill의 열의값 4개를 임의로 선택하여 누락값으로 바꿈
import seaborn as sns
import numpy as np

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,NaN,2.00,Male,No,Sun,Dinner,4
211,NaN,5.16,Male,Yes,Sat,Dinner,4
198,NaN,2.00,Female,Yes,Thur,Lunch,2
176,NaN,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [28]:
# total_bill을 그냥 평균으로하면안됨, 남여의 차이가 있음
count_sex = tips_10.groupby('sex').count()
count_sex

,total_bill,tip,smoker,day,time,size
sex,,,,,,
Male,4,7,7,7,7,7
Female,2,3,3,3,3,3


In [29]:
# 성별을 구분하여 total_bill 열의 데이터를 받아 평균값을 구하는 함수

def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

In [30]:
# 성별을 구분한 total_bill 열의 데이터를 fill_na_mean함수에 전달하여 평균을 구한 뒤 tips_10에 새로운 열로 추가
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
tips_10

,total_bill,tip,sex,smoker,day,time,size,fill_total_bill
24,19.82,3.18,Male,No,Sat,Dinner,2,19.8200
6,8.77,2.00,Male,No,Sun,Dinner,2,8.7700
153,NaN,2.00,Male,No,Sun,Dinner,4,17.9525
211,NaN,5.16,Male,Yes,Sat,Dinner,4,17.9525
198,NaN,2.00,Female,Yes,Thur,Lunch,2,13.9300
176,NaN,2.00,Male,Yes,Sun,Dinner,2,17.9525
192,28.44,2.56,Male,Yes,Thur,Lunch,2,28.4400
124,12.48,2.52,Female,No,Thur,Lunch,2,12.4800
9,14.78,3.23,Male,No,Sun,Dinner,2,14.7800
101,15.38,3.00,Female,Yes,Fri,Dinner,2,15.3800


## 11-3 데이터 필터링

#### 데이터 필터링 사용하기 - filter 메서드

In [34]:
# tips 데이터 불러오기
tips = sns.load_dataset('tips')
tips.shape

(244, 7)

In [35]:
# size 컬럼 확인
tips['size'].value_counts()

2    156
3     38
4     37
5      5
6      4
1      4
Name: size, dtype: int64

In [39]:
# 30번 이상 주문이 있는 데이터 확인
tips_filtered = tips.groupby('size').filter(lambda x : x['size'].count() >= 30)
tips_filtered

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [40]:
tips_filtered.shape

(231, 7)

In [41]:
tips_filtered['size'].value_counts()

2    156
3     38
4     37
Name: size, dtype: int64

## 11-4 그룹 오브젝트

#### 그룹 오브젝트 저장하여 살펴보기

In [42]:
# tips 데이터에서 임의의 10개
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
tips_10

,total_bill,tip,sex,smoker,day,time,size
24,19.82,3.18,Male,No,Sat,Dinner,2
6,8.77,2.00,Male,No,Sun,Dinner,2
153,24.55,2.00,Male,No,Sun,Dinner,4
211,25.89,5.16,Male,Yes,Sat,Dinner,4
198,13.00,2.00,Female,Yes,Thur,Lunch,2
176,17.89,2.00,Male,Yes,Sun,Dinner,2
192,28.44,2.56,Male,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
9,14.78,3.23,Male,No,Sun,Dinner,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [43]:
# groupby 결과값을 보면 object
grouped = tips_10.groupby('sex')
grouped

In [45]:
# groups 속성 출력하면 그룹 오브젝트에 포함된 그룹을 볼수 있음
grouped.groups

{'Male': Int64Index([24, 6, 153, 211, 176, 192, 9], dtype='int64'),
 'Female': Int64Index([198, 124, 101], dtype='int64')}

#### 그룹 오브젝트의 평균 구하기

In [46]:
avgs = grouped.mean()
avgs

,total_bill,tip,size
sex,,,
Male,20.02,2.875714,2.571429
Female,13.62,2.506667,2.000000


In [49]:
# 숫자가 아닌 컬럼은 제외(평균을 낼수 없음으로)
tips.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')

#### 그룹 오브젝트에서 데이터 추출하고 반복하기

In [51]:
# get_group을 사용하여 특정 데이터만 추출
female = grouped.get_group('Female')
female

,total_bill,tip,sex,smoker,day,time,size
198,13.00,2.00,Female,Yes,Thur,Lunch,2
124,12.48,2.52,Female,No,Thur,Lunch,2
101,15.38,3.00,Female,Yes,Fri,Dinner,2


In [52]:
# 반복문에 사용
for sex_group in grouped:
    print(sex_group)

('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


In [53]:
# sex_group의 자세한 정보를 출력
for sex_group in grouped:
    print(f'the type is : {type(sex_group)}')
    print(f'the len is : {len(sex_group)}')
    
    first_element = sex_group[0]
    print(f'the first element is : {first_element}')
    print(f'it has a type of : {type(sex_group[0])}')
    
    second_element = sex_group[1]
    print(f'the second element is : {second_element}')
    print(f'it has a type of : {type(second_element)}')
    
    print('what we have : ')
    print(sex_group)
    
    break

the type is : <class 'tuple'>
the len is : 2
the first element is : Male
it has a type of : <class 'str'>
the second element is :      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2
it has a type of : <class 'pandas.core.frame.DataFrame'>
what we have : 
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2

#### 그룹 오브젝트 계산하고 살펴보기

In [54]:
# sex, time 열을 기준으로 데이터를 그룹화하고 평균값을 구함
bill_sex_time = tips_10.groupby(['sex','time'])
group_avg = bill_sex_time.mean()
group_avg

total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000

In [55]:
# data type 확인
print(type(group_avg))
print(group_avg.columns)

<class 'pandas.core.frame.DataFrame'>
Index(['total_bill', 'tip', 'size'], dtype='object')


In [56]:
# group_avg의 index
group_avg.index

MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])

In [57]:
# 인덱스를 새로 부여
group_method = tips_10.groupby(['sex', 'time']).mean().reset_index()
group_method

,sex,time,total_bill,tip,size
0,Male,Lunch,28.440000,2.560000,2.000000
1,Male,Dinner,18.616667,2.928333,2.666667
2,Female,Lunch,12.740000,2.260000,2.000000
3,Female,Dinner,15.380000,3.000000,2.000000


In [58]:
# as_insex = False로도 같은 결과를 얻음
group_param = tips_10.groupby(['sex', 'time'], as_index=False).mean()
group_param

,sex,time,total_bill,tip,size
0,Male,Lunch,28.440000,2.560000,2.000000
1,Male,Dinner,18.616667,2.928333,2.666667
2,Female,Lunch,12.740000,2.260000,2.000000
3,Female,Dinner,15.380000,3.000000,2.000000
